<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-settings" data-toc-modified-id="Basic-settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic settings</a></span></li><li><span><a href="#POC-English-NoAmb,-step-by-step" data-toc-modified-id="POC-English-NoAmb,-step-by-step-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>POC-English-NoAmb, step-by-step</a></span><ul class="toc-item"><li><span><a href="#Input-parses" data-toc-modified-id="Input-parses-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Input parses</a></span></li><li><span><a href="#Word-space" data-toc-modified-id="Word-space-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Word space</a></span></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Clustering</a></span></li><li><span><a href="#Learn-Grammar" data-toc-modified-id="Learn-Grammar-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Learn Grammar</a></span></li><li><span><a href="#Test-Grammar" data-toc-modified-id="Test-Grammar-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Test Grammar</a></span></li></ul></li><li><span><a href="#POC-English-Amb" data-toc-modified-id="POC-English-Amb-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>POC-English-Amb</a></span></li><li><span><a href="#Child-Directed-Speech" data-toc-modified-id="Child-Directed-Speech-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Child Directed Speech</a></span><ul class="toc-item"><li><span><a href="#20-clusters---PA/PQ-~-71%/51%" data-toc-modified-id="20-clusters---PA/PQ-~-71%/51%-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>20 clusters - PA/PQ ~ 71%/51%</a></span></li><li><span><a href="#Best-results:" data-toc-modified-id="Best-results:-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Best results:</a></span></li><li><span><a href="#Range-3-20" data-toc-modified-id="Range-3-20-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Range 3-20</a></span></li><li><span><a href="#7-clusters:-comparable-with-20-clusters" data-toc-modified-id="7-clusters:-comparable-with-20-clusters-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>7 clusters: comparable with 20 clusters</a></span></li><li><span><a href="#77-clusters" data-toc-modified-id="77-clusters-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>77 clusters</a></span></li></ul></li></ul></div>

# Sparse clustering
Checking first results before commit.  
Relevant data shared on Friday 2018-09-21 via http://langlearn.singularitynet.io/data/clustering_2018/Sparse-Clustering-2018-09-21/

## Basic settings

In [1]:
import os, sys, time
import numpy as np
import pandas as pd
from IPython.display import display
from ull.grammar_learner.utl import UTC
from ull.grammar_learner.read_files import check_dir, check_mst_files
from ull.grammar_learner.pparser import files2links
from ull.grammar_learner.widgets import html_table
from ull.grammar_learner.grammar_inducer import induce_grammar
from ull.grammar_learner.write_files import list2file, save_link_grammar, save_cat_tree
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
from src.grammar_learner_.clustering_ import best_clusters
from src.grammar_learner_.category_learner_ import learn_categories, add_disjuncts, cats2list
from src.grammar_learner_.pqa_table_ import params, pqa_meter
start = time.time()
out_dir = module_path + '/output/sparse-clustering-' + str(UTC())[:10]
check_dir(out_dir, True, 'none')
tmpath = out_dir + '/tmp/'
check_dir(tmpath, True, 'none')
print(UTC(), '\n' + out_dir)

2018-09-24 07:23:41 UTC 
/home/obaskov/py/language-learning/output/sparse-clustering-2018-09-24


In [2]:
kwargs = {
    'left_wall'     :   ''          ,
    'period'        :   False       ,
    'context'       :   2           ,
    'word_space'    :   'sparse'    ,
    'clustering'    :   ('kmeans', 'kmeans++', 10),
    'cluster_range' :   (30,120,3,3), # min, max, step, repeat 
    'cluster_criteria': 'silhouette',
    'cluster_level' :   1           ,
    'grammar_rules' :   2,
    'tmpath'        :   tmpath      , 
    'verbose'       :   'min'       ,
    'template_path' :   'poc-turtle',
    'linkage_limit' :   1000        ,
    'categories_generalization': 'off' }
verbose = 'max'

## POC-English-NoAmb, step-by-step

### Input parses

In [3]:
corpus = 'POC-English-NoAmb'
dataset = 'MST-fixed-manually'
cp = module_path + '/data/POC-English-NoAmb/poc_english_noamb_corpus.txt'
rp = module_path + '/data/POC-English-NoAmb/poc-english_noAmb-parses-gold.txt'
ip, oc, og = params(corpus, dataset, module_path, out_dir, **kwargs)
# print(ip)
# print(oc)
files, re01 = check_mst_files(ip, verbose='none')
kwargs['input_files'] = files
links, re02 = files2links(**kwargs)

### Word space

In [4]:
def co_occurrence_matrix(links):
    wdf = links.groupby('word', as_index=False).sum() \
        .sort_values(by=['count', 'word'], ascending=[False, True])
    if 'djlen' in wdf:
        del wdf['djlen']
    word_list = wdf['word'].tolist()
    word_idx = {word: i+1 for i,word in enumerate(word_list)}
    words = {i: word for word,i in word_idx.items()}

    ldf = links.groupby('link', as_index=False).sum() \
        .sort_values(by=['count', 'link'], ascending=[False, True])
    del ldf['djlen']
    ldf.index = [x for x in range(len(ldf))]
    linx_list = ldf['link'].tolist()
    linx_idx = {link: i for i,link in enumerate(linx_list)}
    linx = {i: link for link,i in linx_idx.items()}
    
    df = links.copy()
    if 'djlen' in wdf:
        del wdf['djlen']
    df['word_id'] = df['word'].apply(lambda x: word_idx[x])
    df['linx_id'] = df['link'].apply(lambda x: linx_idx[x])
    del df['word']
    del df['link']
    df = df[['word_id', 'linx_id', 'count']]
    
    counts = np.zeros((len(words)+1, len(linx)), dtype=np.int16)
    for tpl in df.itertuples():
        counts[tpl[1], tpl[2]] = tpl[3]

    return counts, words, linx

def categorical_distribution(counts, threshold=0.):
    sum = np.sum(counts)
    vsm = np.divide(counts, sum)    # Vector Space Model
    cd = (vsm > threshold).astype(int)  # categorical distribution
    return cd

print(UTC())

2018-09-24 07:23:41 UTC


In [5]:
counts, words, linx = co_occurrence_matrix(links)
print('words:\n', words, '\n\nlinx:\n', linx)
#print('\nCo-occurrence matrix(counts)', type(counts), ':\n', counts)
cd = categorical_distribution(counts, threshold=0.)
#print('\nCategorical distribution (cd)', type(counts), ':\n', cd)

words:
 {1: 'a', 2: 'is', 3: 'now', 4: 'dad', 5: 'mom', 6: 'before', 7: 'cake', 8: 'human', 9: 'likes', 10: 'sausage', 11: 'daughter', 12: 'son', 13: 'food', 14: 'parent', 15: 'was', 16: 'child', 17: 'liked', 18: 'not'} 

linx:
 {0: 'is- & a-', 1: 'is+', 2: 'likes-', 3: 'is-', 4: 'a- & is+', 5: 'human+', 6: 'was-', 7: 'food+', 8: 'parent+', 9: 'was+', 10: 'was- & a-', 11: 'a- & likes+', 12: 'child+', 13: 'daughter+', 14: 'liked-', 15: 'likes+', 16: 'son+', 17: 'dad+', 18: 'mom+', 19: 'liked+', 20: 'cake- & food+', 21: 'cake- & food+ & before+', 22: 'cake- & food+ & now+', 23: 'dad- & human+', 24: 'dad- & human+ & now+', 25: 'dad- & not+ & parent+ & before+', 26: 'dad- & parent+', 27: 'dad- & parent+ & now+', 28: 'dad- & sausage+', 29: 'dad- & sausage+ & before+', 30: 'dad- & sausage+ & now+', 31: 'dad- & son+ & before+', 32: 'daughter- & cake+', 33: 'daughter- & cake+ & now+', 34: 'daughter- & child+', 35: 'daughter- & child+ & now+', 36: 'daughter- & human+', 37: 'daughter- & human+ &

### Clustering

In [6]:
def agglomerative_clustering(cd, words, n_clusters=10, linkage='ward', 
                             silhouette_metric='sqeuclidean'):
    # cd - ndarray(words*disjuncts)
    # words: {id: word}  # legacy compatible: return clusters 
    # linkage: ('ward', 'average', 'complete')
    # silhouette_metric: 'sqeuclidean' ... TODO
    from sklearn.cluster import AgglomerativeClustering
    from sklearn import metrics
    import pandas as pd
    try:
        clustering = AgglomerativeClustering(linkage=linkage, n_clusters=n_clusters)
        clustering.fit(cd)
        labels = clustering.labels_
        silhouette = metrics.silhouette_score(cd, labels, metric=silhouette_metric)
        cdf = 'cdf'
        clusters = {x: [] for x in set(labels)}
        for k,v in words.items():
            clusters[labels[k]].append(v)
        cdf = pd.DataFrame(sorted(set(labels)), columns=['cluster'])
        def cluster_word_list(i):
            return [words[j+1] for j,x in enumerate(labels[1:]) if x==i]
        cdf['cluster_words'] = cdf['cluster'].apply(cluster_word_list)
        cdf.index = range(1, len(cdf)+1)
        def cluster_id(row):
            return 'C' + str(row.name).zfill(2)
        cdf['cluster'] = cdf.apply(cluster_id, axis=1)
        return cdf, silhouette, labels
    except:
        return cdf,  silhouette, ['error']

In [7]:
for n_clusters in range(2, len(cd)-1):
    try:
        cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
        print(n_clusters, 'clusters: silhouette =' , round(silhouette, 3))
    except:
        print('error, silhouette =', silhouette)

2 clusters: silhouette = 0.694
3 clusters: silhouette = 0.37
4 clusters: silhouette = 0.398
5 clusters: silhouette = 0.429
6 clusters: silhouette = 0.443
7 clusters: silhouette = 0.481
8 clusters: silhouette = 0.425
9 clusters: silhouette = 0.498
10 clusters: silhouette = 0.511
11 clusters: silhouette = 0.529
12 clusters: silhouette = 0.561
13 clusters: silhouette = 0.526
14 clusters: silhouette = 0.526
15 clusters: silhouette = 0.421
16 clusters: silhouette = 0.316
17 clusters: silhouette = 0.211


/home/obaskov/miniconda3/envs/ull-dev/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


In [8]:
n_clusters=10
cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
print(n_clusters, 'clusters: silhouette =' , round(silhouette, 3))
print(cdf)

10 clusters: silhouette = 0.511
   cluster                 cluster_words
1      C01            [now, before, not]
2      C02  [human, food, parent, child]
3      C03                    [dad, mom]
4      C04               [daughter, son]
5      C05                       [liked]
6      C06               [cake, sausage]
7      C07                         [was]
8      C08                          [is]
9      C09                       [likes]
10     C10                           [a]


In [9]:
n_clusters=12
cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
print(n_clusters, 'clusters: silhouette =' , round(silhouette, 3))
print(cdf)

12 clusters: silhouette = 0.561
   cluster    cluster_words
1      C01    [before, not]
2      C02  [daughter, son]
3      C03       [dad, mom]
4      C04   [food, parent]
5      C05          [liked]
6      C06  [cake, sausage]
7      C07            [was]
8      C08             [is]
9      C09          [likes]
10     C10              [a]
11     C11            [now]
12     C12   [human, child]


### Learn Grammar

In [10]:
word_space = 'sparse'
algorithm = 'agglomerative_clustering'

def cdf2cats(cdf):
    clusters = cdf
    cats = {}  #80609 dict instead of DataFrame
    cats['cluster'] = ['C0'] + clusters['cluster'].tolist()
    cats['parent'] = [0 for x in cats['cluster']]
    cats['words'] = [[]] + [set(x) for x in clusters['cluster_words'].tolist()]
    if 'disjuncts' in clusters:
        cats['disjuncts'] = [[]] + clusters['disjuncts'].tolist()
        djset = set()
        [[djset.add(y) for y in x] for x in cats['disjuncts']]
        djlist = sorted(djset)
        cats['djs'] = [set([djlist.index(x) for x in y if x in djlist]) \
                       for y in cats['disjuncts']]
    if 'counts' in clusters:
        cats['counts'] = [0] + clusters['counts'].tolist()
    if word_space == 'vectors' or algorithm == 'kmeans':
        cats['quality'] = [0 for x in cats['words']]
        cats['similarities'] = [[0 for y in x] for x in cats['words']]
    else:
        cats['quality'] = [1 for x in cats['words']]
        cats['quality'][0] = 0
        cats['similarities'] = [[1 for y in x] for x in cats['words']]
        cats['similarities'][0] = [0]
    cats['children'] = [0 for x in cats['words']]
    return cats

cats = cdf2cats(cdf)

In [11]:
gen_cats = cats
fat_cats = add_disjuncts(gen_cats, links, verbose)

In [12]:
rules, re07 = induce_grammar(fat_cats, links)

In [13]:
re09 = save_cat_tree(rules, oc, verbose='none')

In [14]:
grammar_rules = 2
re10 = save_link_grammar(rules, og, grammar_rules)

### Test Grammar

In [15]:
%%capture
pa, pq, qa = pqa_meter(re10['grammar_file'], og, cp, rp, **kwargs)

In [16]:
print('parse-ability, parse quality:', 
      str(int(round(pa,0)))+'%', str(int(round(pq,0)))+'%')

parse-ability, parse quality: 46% 39%


## POC-English-Amb

In [17]:
corpus = 'POC-English-Amb'
dataset = 'MST-fixed-manually'
rp = module_path + '/data/POC-English-Amb/MST-fixed-manually/poc-english_ex-parses-gold.txt'
cp = rp  # corpus path = reference path
ip, oc, og = params(corpus, dataset, module_path, out_dir, **kwargs)
files, re01 = check_mst_files(ip, verbose='none')
kwargs['input_files'] = files
links, re02 = files2links(**kwargs)
counts, words, linx = co_occurrence_matrix(links)
cd = categorical_distribution(counts, threshold=0.)
for n_clusters in range(2, len(cd)-1):
    try:
        cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
        print(n_clusters, 'clusters: silhouette =' , round(silhouette, 3))
    except:
        print('error, silhouette =', silhouette)

2 clusters: silhouette = 0.578
3 clusters: silhouette = 0.602
4 clusters: silhouette = 0.466
5 clusters: silhouette = 0.478
6 clusters: silhouette = 0.513
7 clusters: silhouette = 0.516
8 clusters: silhouette = 0.504
9 clusters: silhouette = 0.509
10 clusters: silhouette = 0.356
11 clusters: silhouette = 0.382
12 clusters: silhouette = 0.394
13 clusters: silhouette = 0.209
14 clusters: silhouette = 0.248
15 clusters: silhouette = 0.243
16 clusters: silhouette = 0.256
17 clusters: silhouette = 0.271
18 clusters: silhouette = 0.28
19 clusters: silhouette = 0.29
20 clusters: silhouette = 0.3
21 clusters: silhouette = 0.303
22 clusters: silhouette = 0.306
23 clusters: silhouette = 0.309
24 clusters: silhouette = 0.313
25 clusters: silhouette = 0.316
26 clusters: silhouette = 0.32
27 clusters: silhouette = 0.318
28 clusters: silhouette = 0.28
29 clusters: silhouette = 0.257
30 clusters: silhouette = 0.233
31 clusters: silhouette = 0.233
32 clusters: silhouette = 0.233
33 clusters: silhouett

/home/obaskov/miniconda3/envs/ull-dev/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:205: RuntimeWarning: invalid value encountered in true_divide
  sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)


In [18]:
n_clusters = 7
cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
print(n_clusters, 'clusters: silhouette index =' , round(silhouette, 3))
cats = cdf2cats(cdf)
for x in cats['words'][1:]: print(x)
gen_cats = cats
fat_cats = add_disjuncts(gen_cats, links, verbose)
rules, re07 = induce_grammar(fat_cats, links)
re09 = save_cat_tree(rules,  oc, verbose='none')
grammar_rules = 2
re10 = save_link_grammar(rules, og, grammar_rules)

7 clusters: silhouette index = 0.516
{'wood', 'telescope', 'human', 'tool', 'liked', 'board', 'parent', 'on', 'before', 'chalk', 'directors', 'be', 'to', 'his', 'writes', 'knocked', 'of', 'sees', 'not', 'sausage', 'was', 'are', 'sawed', 'the', 'child', 'hammer', 'wants', 'food', 'likes', 'now', 'binoculars', 'cake', 'her'}
{'has', 'with'}
{'mom', 'dad'}
{'son', 'daughter'}
{'a'}
{'is'}
{'saw'}


In [19]:
%%capture
pa, pq, qa = pqa_meter(re10['grammar_file'], og, cp, rp, **kwargs)

In [20]:
print('parse-ability, parse quality:', 
      str(round(pa,1))+'%', str(round(pq,1))+'%')

parse-ability, parse quality: 84.6% 69.5%


**Baseline**:  
cDRKd: 90%/86%, 18 rules  
dILEd: 36%/32%, 37 rules (no generalization)

In [21]:
#STOP

## Child Directed Speech

In [22]:
corpus = 'CDS-caps-br-text+brent9mos'
dataset = 'LG-English'
rp = module_path + '/data/CDS-caps-br-text+brent9mos/LG-English'
cp = rp  # corpus path = reference_path :: use 'gold' parses as test corpus
ip, oc, og = params(corpus, dataset, module_path, out_dir, **kwargs)
files, re01 = check_mst_files(ip, verbose='none')
kwargs['input_files'] = files
links, re02 = files2links(**kwargs)
counts, words, linx = co_occurrence_matrix(links)
cd = categorical_distribution(counts, threshold=0.)
print(UTC())

2018-09-24 07:23:44 UTC


### 20 clusters - PA/PQ ~ 71%/51%

In [23]:
n_clusters = 20
cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
print(n_clusters, 'clusters: silhouette index =' , round(silhouette, 3))
cats = cdf2cats(cdf)
# for x in cats['words'][1:]: print(x)
gen_cats = cats
fat_cats = add_disjuncts(gen_cats, links, verbose)
rules, re07 = induce_grammar(fat_cats, links)
re09 = save_cat_tree(rules,  oc, verbose='none')
grammar_rules = 2
re10 = save_link_grammar(rules, og, grammar_rules)
print(UTC())

20 clusters: silhouette index = 0.889
2018-09-24 07:27:09 UTC


In [24]:
%%capture
pa, pq, qa = pqa_meter(re10['grammar_file'], og, cp, rp, **kwargs)

In [25]:
print('parse-ability, parse quality:', 
      str(round(pa,1))+'%', str(round(pq,1))+'%')
print(UTC())

parse-ability, parse quality: 70.7% 51.4%
2018-09-24 07:31:23 UTC


### Best results:
 3 rules: 71.9%/49.7% (silhouette index 0.978)  
 7 rules: 71.7%/50.6% (silhouette index 0.964)  
20 rules: 70.7%/51.4% (silhouette index 0.889)  
77 rules: 63.6%/49.3% (silhouette index 0.529)  
**Previous tests 2018-08-14:**  
cDRKc: 71%/50% 78 rules;  
dILEd: 40%/37% 2980 rules

### Range 3-20

In [26]:
for n_clusters in range(3, 22, 2):
    try:
        cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
        print(n_clusters, 'clusters: silhouette =' , round(silhouette, 3))
    except:
        print('error, silhouette =', silhouette)

3 clusters: silhouette = 0.978
5 clusters: silhouette = 0.964
7 clusters: silhouette = 0.907
9 clusters: silhouette = 0.906
11 clusters: silhouette = 0.901
13 clusters: silhouette = 0.892
15 clusters: silhouette = 0.894
17 clusters: silhouette = 0.894
19 clusters: silhouette = 0.892
21 clusters: silhouette = 0.89


### 7 clusters: comparable with 20 clusters

In [27]:
n_clusters = 7
cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
print(n_clusters, 'clusters: silhouette index =' , round(silhouette, 3))
cats = cdf2cats(cdf)
for x in cats['words'][1:]: print(x)
gen_cats = cats
fat_cats = add_disjuncts(gen_cats, links, verbose)
rules, re07 = induce_grammar(fat_cats, links)
re09 = save_cat_tree(rules,  oc, verbose='none')
re10 = save_link_grammar(rules, og, grammar_rules)
print(UTC())

7 clusters: silhouette index = 0.907
{"let's", 'he', 'no', 'back', 'on', 'mommy', 'this', 'my', 'his', 'wanna', 'oh', 'do', 'so', 'out', 'right', 'your', 'see', 'yeah', 'say', 'i', 'over', 'up', 'here', 'okay', 'that', 'down', 'little', 'baby', 'what', "don't", 'all', 'me', 'one', 'more', 'in', 'get', 'good', 'go', 'they', 'some', 'there', 'it', 'want', 'have', 'now', 'those', 'him', 'like', 'big', 'we', 'them', 'just', 'something'}
{'the', 'a'}
{'babes', 'friday', 'express', 'carrying', 'sugar', 'cried', 'draw', 'bike', 'father', 'licking', 'lever', 'hardheaded', 'sauce', 'man', 'asking', 'gus', 'zip', 'works', 'fake', 'baebae', 'fire', 'stupid', 'drop', 'spit', 'throat', 'airplane', 'listen', 'sweetie', 'trucks', 'cool', 'peep', 'belly', 'mandy', 'earth', 'peppers', 'pouch', 'through', 'panting', 'body', 'infection', 'drinkie', 'prayers', 'eleven', 'dragon', 'alone', 'tracks', 'untie', 'chance', 'scrubbie', 'barbells', 'pampers', 'spinning', 'pinch', 'clear', 'kissie', 'suzie', 'unde

2018-09-24 08:09:25 UTC


In [28]:
%%capture
pa, pq, qa = pqa_meter(re10['grammar_file'], og, cp, rp, **kwargs)

In [29]:
print('parse-ability, parse quality:', 
      str(round(pa,1))+'%', str(round(pq,1))+'%')
print(UTC())

parse-ability, parse quality: 71.7% 50.6%
2018-09-24 08:42:44 UTC


### 77 clusters

In [30]:
n_clusters = 77
cdf, silhouette, labels = agglomerative_clustering(cd, words, n_clusters)
print(n_clusters, 'clusters: silhouette index =' , round(silhouette, 3))
cats = cdf2cats(cdf)
for x in cats['words'][1:]: print(x)
gen_cats = cats
fat_cats = add_disjuncts(gen_cats, links, verbose)
rules, re07 = induce_grammar(fat_cats, links)
re09 = save_cat_tree(rules,  oc, verbose='none')
re10 = save_link_grammar(rules, og, grammar_rules)
print(UTC())

77 clusters: silhouette index = 0.529
{'babes', 'express', 'carrying', 'sugar', 'cried', 'draw', 'bike', 'father', 'licking', 'lever', 'hardheaded', 'sauce', 'asking', 'zip', 'works', 'fake', 'baebae', 'fire', 'stupid', 'drop', 'spit', 'throat', 'airplane', 'listen', 'sweetie', 'trucks', 'cool', 'peep', 'belly', 'earth', 'peppers', 'pouch', 'through', 'panting', 'body', 'infection', 'drinkie', 'prayers', 'eleven', 'alone', 'tracks', 'untie', 'chance', 'scrubbie', 'barbells', 'pampers', 'spinning', 'pinch', 'clear', 'kissie', 'suzie', 'underwear', 'destruction', 'agree', 'problem', 'pet', 'yelling', 'scratching', 'nearly', 'circle', 'thirty', 'juices', 'swings', 'cooking', 'ourself', 'pulling', 'august', 'touched', 'didey', 'bless', 'oatmeal', 'backordered', 'rearrange', 'scenes', 'happen', 'amelia', 'rahab', 'grate', 'unicorn', 'pages', 'lake', 'gum', 'n@l', 'ruffle', 'toilet', 'riders', 'lordy', 'weekend', 'turkey', 'boomerang', 'may', 'zoo', 'mooshas', 'papa', 'beach', 'beep', 'frost

2018-09-24 08:46:24 UTC


In [31]:
%%capture
pa, pq, qa = pqa_meter(re10['grammar_file'], og, cp, rp, **kwargs)

In [32]:
print('parse-ability, parse quality:', 
      str(round(pa,1))+'%', str(round(pq,1))+'%')
print(UTC())

parse-ability, parse quality: 63.6% 49.3%
2018-09-24 08:49:17 UTC
